In [ ]:
import pandas as pd
import numpy as np
from pandas.core.api import notnull
import os
import re
import csv 
import json
import glob
from google.colab import drive

# Prepare dataset

In [ ]:
""" using google.colab to connect the raw data file.
    E.g., "/content/drive/MyDrive/Malaxy.

    Parameters:
    folderName (str): The raw data file name like course,instructor,etc.
    
    Returns:
    RawData: dataframe for future use
"""
def PrepareDataset(folderName):
    drive.mount('/content/drive')
    workdir = '/content/drive/MyDrive/Malaxy/'+folderName
    RawData = pd.read_excel( os.path.join( workdir, 'RawData.xlsx' ), header=0 )
    return RawData


""" This method is for common merging use: 
    create a list contains different dataframe
    it will auto loop it and merge them

    Parameters:
    tablelist: a list of dataframe contains what we want to merge
    method: left,right or inner join
    collumnName: the connection collumn

    Returns:
    outer_merged: merged dataframe
"""
def merge(tablelist,method, collumnName):
  outer_merged = tablelist[0]
  for i in range(len(tablelist)-1):
    if method == "inner":
      outer_merged = pd.merge(outer_merged, tablelist[i+1], how="inner", on=[collumnName])
    elif method == "left":
      outer_merged = pd.merge(outer_merged, tablelist[i+1], how="left", on=[collumnName])
    else:
      outer_merged = pd.merge(outer_merged, tablelist[i+1], how="right", on=[collumnName])
  return outer_merged

  


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Cleaning Course

In [ ]:
""" This method is design for cleaning dirty course data: 
    E.g., take out 506 in CS506 and put it in the other column alone.

    Parameters:
    RawData (str): The dataframe created
    ColumnName1(str): original column
    ColumnName2(str): new column 

    Returns:
    RawData: dataframe 
"""

def DivColumnName(RawData,ColumnName1,ColumnName2):
    RawData[ColumnName2] = RawData[ColumnName1]
    for i in range(len(RawData[ColumnName1])):
        RawData[ColumnName2][i] = re.findall(r'(.*[^\d+$])',RawData[ColumnName1][i])
    return RawData


# 2. Cleaning instructor

In [ ]:
""" This method is design for cleaning dirty instructor data: 
    E.g., i have lots of instructor info from different semester
    1. get all csv file under this folder 
    2. add semester in each file
    3. export and replace original files

    Parameters:
    folderName (str): The raw data file name like course,instructor,etc.

    Returns:
    True if csv file with semester added successfully.
"""

def GetAllCSV(folderName):
     workDir = '/content/drive/MyDrive/Malaxy'+ folderName
     # create a list to get all csv files name under this folder using os
     fileList = os.listdir(workDir)
     # adding path to all file names
     filePath = np.array([workDir + file for file in fileList if file.endswith('.csv')],
                        dtype=object)
    # and create dataframe
     GetAllCSV = pd.DataFrame(filePath.reshape(len(filePath), 1))
     for path in GetAllCSV[0]:
        df = pd.read_csv(path, encoding = "UTF-16", sep='\t')
        #semester: 2022-2023 spring
        sem_elems = path.split('/')[-1][:-4].split(' ')
        #change it to : Spring 2022-2023
        semester = sem_elems[1].capitalize() + ' ' + sem_elems[0]
        df["SemesterName"] = [semester]*len(df)
        df.to_csv(path)
        return True



In [ ]:
# This is for basic cleaning dirty instructor data: 
# E.g., meged data, drop duplicateds, rename Instructors' name 
# and output a cleaned Instructor table

# Sometimes I used different ways of them to clean data, there is no fixed procedure


workDir = '/content/drive/MyDrive/Malaxy/instructor'
#merged instructor data
Instructordata= pd.concat(map(pd.read_csv, glob.glob(os.path.join(workDir, '*.csv'))), ignore_index= True)
Instructordata= pd.DataFrame(Instructordata,columns = ["Instructor Name","Class (Section)","SemesterName"])
INSTRUCTORTABLE= pd.DataFrame(Instructordata,columns = ["Instructor Name",'ProfessorID'])
INSTRUCTORTABLE = INSTRUCTORTABLE.drop_duplicates()
INSTRUCTORTABLE = INSTRUCTORTABLE.reset_index(drop=True)
#switch first and last name 
INSTRUCTORTABLE['Professor Name'] = INSTRUCTORTABLE['Instructor Name'].str.split(',').str[1] + ' ' + INSTRUCTORTABLE['Instructor Name'].str.split(',').str[0]
INSTRUCTORTABLE = INSTRUCTORTABLE.drop(columns=['Instructor Name'])
INSTRUCTORTABLE.to_csv( os.path.join( workDir, 'Instructordata.csv'),encoding="utf-8",sep='|')


# 3. Cleaning Section table

In [ ]:
# This is for basic cleaning dirty Section data: 
# E.g., grouby data, drop duplicateds, rename Instructors' name 
# and output a cleaned Instructor table

# Sometimes I used different ways of them to clean data, there is no fixed procedure

#groupby semester Name and ID
Instructordata['SemesterID']=pd.factorize(Instructordata['SemesterName',"SemesterID"])[0]+1
Instructordata = Instructordata.drop(columns=["SemesterName", "SemesterID"])
# drop duplicate
Instructordata.drop_duplicates().sort_values("Class ")
sectiontable = Instructordata
#cleaning class collumn
sectiontable["Class"] = sectiontable["Class "].str.strip()
sectiontable = sectiontable.drop(columns=["Class "])
sectiontable = sectiontable.drop_duplicates()
workDir = '/content/drive/MyDrive/Malaxy/Section'
sectiontable.to_csv( os.path.join( workDir, 'sectiontable.csv'),encoding="utf-8",sep='|')


# 4. Cleaning grade

In [ ]:
# This is for basic cleaning dirty instructor data: 
# E.g., meged data, drop duplicateds, rename Instructors' name 
# and output a cleaned Instructor table

# Sometimes I used different ways of them to clean data, there is no fixed procedure

#Use courselist as reference table
workdir="/content/drive/MyDrive/Malaxy/course"
workdir2="/content/drive/MyDrive/Malaxy/Grade"

    #get all abbreviation of courses
courselist = courselist.drop_duplicates()
courselist= courselist.reset_index(drop=True)
courselist["college name"] = courselist["college name"].str.strip()
Gradedata = pd.concat(map(pd.read_csv, glob.glob(os.path.join(workdir2, '*.csv'))), ignore_index= True)
Gradedata.rename(columns = {'Unnamed: 0.1':'college name','Unnamed: 2':'CourseTitle'},inplace = True)
table = [Gradedata,courselist]
mergedcollege = merge(table,"inner","college name")
mergedcollege.rename(columns = {'Unnamed: 1':'sectionid'},inplace = True)
mergedcollege = mergedcollege.drop(columns=['Unnamed: 0', 'Unnamed: 3','Unnamed: 4','college name',"sectionid","CourseTitle",])
mergedcollege["CourseWithCategoryID"] = mergedcollege["abbreviation"] + " "+ mergedcollege["sectionid"].map(str) 
mergedcollege = mergedcollege.drop(columns=["abbreviation"])
mergedcollege = mergedcollege.dropna(subset=['GPA '])
mergedcollege.rename(columns = {'GPA ':'GPA'},inplace = True)
mergedcollege.drop_duplicates(subset=['SemesterName','CourseWithCategoryID'],keep='first', inplace=True)

    # connect table with section id
sectionwithid = pd.read_csv( os.path.join( workdir, 'sorry.csv' ), header=0 )
sectionwithid1 = pd.DataFrame(sectionwithid, columns = ['CourseName','SectionId'])
sectionwithid1.rename(columns = {'CourseName':'CourseWithCategoryID'},inplace = True)
table = [mergedcollege,sectionwithid1]
mergedcollege = merge(table,"left","CourseWithCategoryID")
mergedcollege = mergedcollege.dropna(subset=['SectionId'])

mergedcollege.rename(columns = {'GPA':'Gpa', 'A':'GradeCountA', 'AB':'GradeCountAb', 'B':'GradeCountB',
                                    'BC':'GradeCountBc', 'C':'GradeCountC', 'D':'GradeCountD', 
                                    'F':'GradeCountF', 'S/SD/CR':'GradeCountSd', 'N/U/UD':'GradeCountUd',
        'Other':'GradeCountOther', 'Total ':'GradeCountTotal', 'SemesterName':'SemesterName', 
        'CourseWithCategoryID':'CourseWithCategoryID'},inplace = True)

mergedcollege.to_csv( os.path.join( workdir, 'gradd.csv'),encoding="utf-8",sep='|')

In [ ]:
#check differences
# sorted(list(set(Gradedata["college name"].to_list())))[:10]
# sorted(list(set(courselist["college name"].to_list())))[:10]